In [1]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-71e759b7-e61b-46b6-badf-6e19e5dfd392;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 240ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/28 05:04:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/28 05:04:41 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from graphframes import GraphFrame

In [ ]:
window = Window.partitionBy("relationship").orderBy("timestamp")
df = readFirstEvents().withColumn("relationship", sort_array(array("actorID", "objectID")) )  \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1).cache()# \
                #.drop('rank').cache()

In [ ]:
df = df.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0))

In [ ]:
df.count()

In [ ]:
df.limit(10).toPandas()

In [3]:
window = Window.partitionBy("relationship").orderBy("timestamp")
df_2 = readFirstEvents().sort("timestamp").withColumn("relationship", sort_array(array("actorID", "objectID")) )  \
                .withColumn("row_num", row_number().over(window)) \
                .filter(col("row_num") == 1).cache()

 5:04AM UTC on Mar 28, 2023 --- read time: 5.133359670639038 seconds ---


In [ ]:
df_2.count()

In [4]:
df_23 = df_2.where(col('event_day') == 23).cache()

In [ ]:
df_23.count()

In [ ]:
df.printSchema()

In [ ]:
df_2 = df_2.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0)) \
        .drop('row_num','relationship')

In [ ]:
print("image path: "+str(df.select("image_path").distinct().count()))
print("image path: "+str(df.select("parent_image_path").distinct().count()))

In [ ]:
df_100k = df.limit(100000).cache()
df_2_100k = df_2.limit(100000).cache()

In [ ]:
df.count()

In [ ]:
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_100k.selectExpr('actorID as id').distinct()
dst_vertices = df_100k.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_100k.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)

In [ ]:
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_2_100k.sort('timestamp').selectExpr('actorID as id').distinct()
dst_vertices = df_2_100k.sort('timestamp').selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_2_100k.sort('timestamp').selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

edges.sort('timestamp')

# Create GraphFrame
g_2 = GraphFrame(vertices, edges)

In [ ]:
# display the vertices
print("Vertices:")
g.vertices.show()

In [ ]:
# display the edges
print("Edges:")
g.edges.show(5)

In [ ]:
g.inDegrees.show(5)

In [ ]:
# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2_1 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)")
#motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
motifs6_1 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

In [ ]:
motifs2_1.count()

In [ ]:
motifs6_1.count()

In [ ]:
# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2_2 = g_2.find("(a)-[e1]->(b); (b)-[e2]->(c)")
#motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
motifs6_2 = g_2.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

In [ ]:
print(motifs2_2.count())
print(motifs6_2.count())

In [ ]:
#filter the motif2 to check the theory of timestamp increasing
cp3 = motifs2_2.filter("e1.timestamp < e2.timestamp")# and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")

In [ ]:
cp3.limit(2).toPandas()

In [ ]:
#explode it and see what you got
df_transp = cp3.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])

#drop all vertices
df_transp = df_transp.drop('a','b','c')

#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    ELSE 'e2' END''').alias("event"),
    "col"
).orderBy("Trace","event")

#explode columns
df_onehot = stacked_df.select(*stacked_df.columns, "col.*").drop('col')

In [ ]:
df_onehot.limit(6).toPandas()

In [ ]:
# filter paths to only those where all edges are connected
cp_1 = motifs6_1.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")
cp_2 = motifs6_2.filter('''e1.timestamp <= e2.timestamp and 
                            e2.timestamp <= e3.timestamp and 
                            e3.timestamp <= e4.timestamp and 
                            e4.timestamp <= e5.timestamp and 
                            e5.timestamp <= e6.timestamp''')

In [ ]:
print(cp_1.count())
print(cp_2.count())

In [ ]:
motifs6_2.limit(1).toPandas()

In [ ]:
cp_2.limit(12).toPandas()

In [5]:
#run transp explode on both and see what we have. first run it on both motif and then filter. this is on fully de-duped
#data.
%run trace_encode.ipynb

In [ ]:
te_2a = transp_expl(motifs6_2)
te_2b = transp_expl(cp_2)
#the results of the above are the same, no need to filter

In [ ]:
te_2b.limit(12).toPandas()

In [9]:
#now run everything on all the data from 23Sep to see how long it takes
start_time = time.time()
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_23.selectExpr('actorID as id').distinct()
dst_vertices = df_23.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')
# Create GraphFrame
g = GraphFrame(vertices, edges)

#motifs
motifs6_23 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")
#end time
print("found connections: "+ str(time.time() - start_time))
#filter on time to generate final connected paths
cp = motifs6_23.filter('''e1.timestamp <= e2.timestamp and 
                            e2.timestamp <= e3.timestamp and 
                            e3.timestamp <= e4.timestamp and 
                            e4.timestamp <= e5.timestamp and 
                            e5.timestamp <= e6.timestamp''').cache()

#transpose and explode
te_23 = transp_expl(cp).cache()

#end time
print("elapsed time: "+ str(time.time() - start_time))

found connections: 1.3243827819824219


elapsed time: 2.5701472759246826


In [10]:
te_23.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)



In [ ]:
# get the 6 edges in each path and convert to a pandas dataframe
connected_paths.select("e1").limit(1).collect()

In [ ]:
type(connected_paths)

In [ ]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")

In [ ]:
connected_paths = spark.read.parquet(f"{s3_url_trusted}/prod/graph/test").cache()

In [ ]:
connected_paths.limit(5).toPandas()

In [ ]:
connected_paths.count()

In [ ]:
# using split and applying in ml with pypark
train, test, val = connected_paths.randomSplit([0.80, 0.15, 0.05], seed=12345)

In [ ]:
val.count()

In [ ]:
# create new dataframe to union and create duplicate rows
df_dup = df.withColumn('id', lit(None).cast(StringType()))

In [ ]:
df_dup.limit(5).toPandas()

In [ ]:
df_new = df.union(df_dup).sort("timestamp").cache()
df.unpersist()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
# len(df_new.select('relationship').distinct().collect())

In [ ]:
tot = df_new.count()

In [ ]:
df

In [ ]:
df_ratios = df.groupBy("object", "event_day", "malicious") \
              .count() \
              .withColumnRenamed('count', 'cnt_per_group') \
              .withColumn('perc_of_count_total', (col('cnt_per_group') / tot) * 100 ) \
              .sort("event_day", "malicious",  "perc_of_count_total", "object") \
              .cache()

In [ ]:
df_ratios.limit(20).toPandas()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")